In [50]:
from scrapy import signals
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing
import datetime
import collections
import urllib
import collections
from scrapy import Request as Request

In [51]:
business_data=collections.namedtuple('business_data','link,title,status')

In [52]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(get_project_settings())
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

In [53]:
class CanberraBusinessSpider(scrapy.Spider):
    name = "CanberraBusinessSpider"
    allowed_domains = ["www.seekbusiness.com.au"]
    start_urls = ['https://www.seekbusiness.com.au/businesses-for-sale/in-canberra-act-2601?rad=50']

    def parse(self, response):
        next_page=response.xpath('//a[contains(@class,"next")]/@href').extract()[0]
        next_page_link=urllib.basejoin(response.url,next_page)
        #print(next_page_link)
        #yield Request(next_page_link,callback=self.parse)
        
        business_tags=response.xpath('//div[contains(@class,"sr-l") and @onclick]')
        for business in business_tags:
            title=business.xpath('div[@class="t"]/a/text()').extract()[0]
            link=business.xpath('div[@class="t"]/a/@href').extract()[0]
            full_link=urllib.basejoin(response.url,link)
            contents=business.xpath('div[@class="m-c"]/div[@class="det-c"]/div[@class="smry"]/text()').extract()
            full_content='\n'.join(contents)
            print title,full_link,full_content
        """
        business_tags=response.xpath('//a[contains(@href,"business-listing") and count(@*)=1]')
        for invidual_business in business_tags:
            title=invidual_business.xpath('text()').extract()[0]
            link=invidual_business.xpath('@href').extract()[0]
            full_link=urllib.basejoin(response.url,link)
            yield {title:business_data(full_link,title,'not yet')}
            #business={'title':title,'link':full_link}
            #print(business)
        #print(business_tags)
        """
        

In [54]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(CanberraBusinessSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        print item
        
if __name__=="__main__":
    main()

2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-19 09:58:04 [scrapy] INFO: Overridden settings: {}
2017-01-19 09:58:04 [scrapy] INFO: Overridden settings: {}
2017-01-19 09:58:04 [scrapy] INFO: Overridden settings: {}
2017-01-19 09:58:04 [scrapy] INFO: Overridden settings: {}
2017-01-19 09:58:04 [scrapy] INFO:

Rent Roll Sale Canberra/Queanbeyan Region (75-100 properties) https://www.seekbusiness.com.au/business-listing/businesses/291164?s=0 Rent Roll Sale Canberra/Queanbeyan Region (75-100 properties)
                

            
CafeCoffeeShop - Canberra https://www.seekbusiness.com.au/business-listing/businesses/265305?s=0 Cafe -    Takeaway  -  Franchise - Canberra
                

            
Mo's Mobiles is looking for Licensed Dealers to operate existing Vodafone stores https://www.seekbusiness.com.au/business-listing/businesses/281849?s=0 Mo’s Mobiles is Vodafone’s largest exclusive dealer. We are looking for Licensees to operate a number of our existing stores.

                

            
Gozleme King Franchise : Turkish Street Food & Cafe https://www.seekbusiness.com.au/business-listing/businesses/270622?s=0 Gözleme King is  offering premium franchise opportunities located in major shopping centers , busy high street's and major CBD's.  Contact us now to find out more.
     

2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Closing spider (finished)
2017-01-19 09:58:06 [scrapy] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 268,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 29834,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 1, 18, 22, 58, 6, 280226),
 'log_count/D